# Name(s)
**Ethan Moore and Ajay Patel**

**Instructions:** Pair programming assignment. Submit only a single notebook unless you deviate significantly after lab on Thursday. If you submit individually, make sure you indicate who you worked with originally. Make sure to include your first and last names.

# Decision Trees

## Preface
(Courtesy of Dr. Alex Dekhtyar)

The core objective of Knowledge Discovery in Data/Data Mining/Machine Learning methods is to provide efficient algorithms for gaining insight from data. CSC 466 primarily studies the methods and the algorithms that enable
such insight, and that specifically take this insight above and beyond traditional statistical analysis of data (more
about this — later in the course).
However, the true power of KDD/DM/ML methods that we will study in this course is witnessed only when
these methods are applied to actually gain insight from the data. As such, in this course, the deliverables for your
laboratory assignments will be partitioned into two categories:

1. KDD Method implementation. In most labs you will be asked to implement from scratch one or more
KDD method for producing a special type of insight from data. This part of the labs is similar to your other
CS coursework - you will submit your code, and, sometimes, your tests and/or output.

2. Insight, a.k.a., data analysis. For each lab assignment we will provide one or more datasets for your
perusal, and will ask you to perform the analysis of these datasets using the methods you implemented. The
results of this analysis, i.e., the insight, are as important for successful completion of your assignments, as
your implementations. Most of the time, you will be asked to submit a lab report detailing your analysis,
and containing the answers to the questions you are asked to study.
The insight portion of your deliverables is something that you may be seeing for the first time in your CS
coursework. It is not an afterthought in your lab assignments. Your grade will, in no small part, depend on
the results of your analysis, and the writing quality on your report. This lab assignment, and further assignments
will include detailed insturctions on how to prepare reports, and we will discuss report writing several times as
the course progresses.

## Lab Assignment

This is a pair programming assignment. I strongly
discourage individual work for this (and other team/pair programming) lab(s), even if you think you can do it
all by yourself. Also, this is a pair programming assignment, not a ”work in teams of two” assignment. Pair
programming requires joint work on all aspects of the project without delegating portions of the work to individual
1
team members. For this lab, I want all your work — discussion, software development, analysis of the results,
report writing — to be products of joint work.
Students enrolled in the class can pair with other students enrolled in the class. Students on the waitlist can
pair with other students on the waitlists. In the cases of ”odd person out” situations, a team of three people can
be formed, but that team must (a) ask and answer one additional question, and (b) work as a pair would, without
delegation of any work off-line.

For this lab, we are going to implement a variation of the C4.5 decision tree with statistical pruning. C4.5 provides several improvements over ID3 though the base structure is very similar. Note that there are different methods for pruning a decision tree. Here we will use statistical confidence intervals. If you have trouble getting started, don't forget Marsland has sample code on his website that corresponds to the book.

We will start with our titanic dataset first.

In [1]:
import pandas as pd
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

titanic_df = pd.read_csv(
    "https://raw.githubusercontent.com/dlsun/data-science-book/master/data/titanic.csv"
)
titanic_df.head()

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest
0,1,1,"Allen, Miss. Elisabeth Walton",female,29.0000,0,0,24160,211.3375,B5,S,2,NaN,"St Louis, MO"
1,1,1,"Allison, Master. Hudson Trevor",male,0.9167,1,2,113781,151.5500,C22 C26,S,11,NaN,"Montreal, PQ / Chesterville, ON"
2,1,0,"Allison, Miss. Helen Loraine",female,2.0000,1,2,113781,151.5500,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON"
3,1,0,"Allison, Mr. Hudson Joshua Creighton",male,30.0000,1,2,113781,151.5500,C22 C26,S,NaN,135.0,"Montreal, PQ / Chesterville, ON"
4,1,0,"Allison, Mrs. Hudson J C (Bessie Waldo Daniels)",female,25.0000,1,2,113781,151.5500,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON"


We only need a few columns, and I will also perform some preprocessing for you:

In [2]:
features = ['pclass','survived','sex','age']
titanic_df = titanic_df.loc[:,features]
display(titanic_df)
titanic_df.loc[:,'pclass']=titanic_df['pclass'].fillna(titanic_df['pclass'].mode()).astype(int)
titanic_df.loc[:,'age']=titanic_df['age'].fillna(titanic_df['age'].median())
titanic_df.loc[:,'age']=(titanic_df['age']/10).astype(str).str[0].astype(int)*10
titanic_df

,pclass,survived,sex,age
0,1,1,female,29.0000
1,1,1,male,0.9167
2,1,0,female,2.0000
3,1,0,male,30.0000
4,1,0,female,25.0000
...,...,...,...,...
1304,3,0,female,14.5000
1305,3,0,female,NaN
1306,3,0,male,26.5000
1307,3,0,male,27.0000


,pclass,survived,sex,age
0,1,1,female,20
1,1,1,male,0
2,1,0,female,0
3,1,0,male,30
4,1,0,female,20
...,...,...,...,...
1304,3,0,female,10
1305,3,0,female,20
1306,3,0,male,20
1307,3,0,male,20


In [3]:
titanic_df.describe()

,pclass,survived,age
count,1309.000000,1309.000000,1309.000000
mean,2.294882,0.381971,24.385027
std,0.837836,0.486055,13.387598
min,1.000000,0.000000,0.000000
25%,2.000000,0.000000,20.000000
50%,3.000000,0.000000,20.000000
75%,3.000000,1.000000,30.000000
max,3.000000,1.000000,80.000000


## Exercise 1
Construct a function that calculates the entropy of a set (Pandas Series Object).
<pre>
def entropy(y):
  ???
  return e
</pre>

In [4]:
import numpy as np
def entropy(y):
    e = 0
    for i in y.value_counts(normalize=True):
        e += i * np.log2(i)
    return abs(e)

In [5]:
entropy(titanic_df['survived'])

0.959422170862815

In [6]:
# YOUR SOLUTION HERE
display(entropy(titanic_df['survived']))
display(entropy(pd.Series([0,0,0,1,1,1])))
display(entropy(pd.Series([0,0,0])))

0.959422170862815

1.0

0.0

## Exercise 2
Now write a function that calculates the information gain after splitting with a specific variable (Equation 12.2 from Marsland).
<pre>
def gain(y,x):
  ???
  return g
</pre>

In [7]:
def gain(y, x):
    g = 0
    for i in x.unique():
#         print(i)
        data = x[x==i]
        datay = y[x==i]
        e = 0
        for j in datay.value_counts(normalize=True):
            e += j * np.log2(j)
        g += abs(e) * (len(data)) / len(x)
        
    return entropy(y) - g

In [8]:
# YOUR SOLUTION HERE
display(gain(titanic_df['survived'],titanic_df['sex']))
display(gain(titanic_df['survived'],titanic_df['pclass']))
display(gain(titanic_df['survived'],titanic_df['age']))

0.205504872720076

0.0704074128460409

0.01786395614222558

## Exercise 3
C4.5 actually uses the gain ratio which is defined as the information gain "normalized" (divided) by the entropy before the split. You have written everything you need here. Just put it together.

<pre>
def gain_ratio(y,x):
  ???
  return gr
</pre>

In [9]:
def gain_ratio(y, x):
#     print("Entropy y: ", entropy(y))
    gr = gain(y, x) / entropy(y)
    return gr

In [10]:
# YOUR SOLUTION HERE
display(gain_ratio(titanic_df['survived'],titanic_df['sex']))
display(gain_ratio(titanic_df['survived'],titanic_df['pclass']))
display(gain_ratio(titanic_df['survived'],titanic_df['age']))

0.21419650177071065

0.07338522600819515

0.01861949482172212

## Exercise 4
Define a function that chooses the column to split the tree out of possible columns in the dataframe.

<pre>
def select_split(X,y):
   ???
   return column,gain
</pre>

In [11]:
titanic_df.columns

Index(['pclass', 'survived', 'sex', 'age'], dtype='object')

In [12]:
def select_split(X, y):
    max_col = 0
    for i in X.columns:
        gr = gain_ratio(y, X[i])
        if gr > max_col:
            max_col = gr
            best_col = i
            
    return best_col, max_col

In [13]:
# YOUR SOLUTION HERE
select_split(titanic_df.drop('survived',axis=1),titanic_df['survived'])

('sex', 0.21419650177071065)

In [14]:
select_split(titanic_df.drop('survived',axis=1),titanic_df['survived'])[0]

'sex'

## Exercise 5
Now put it all together and construct a function called make_tree that returns a tree in a proprietary format of your choosing. Mine is a dictionary, but do whatever you want as long as it works :)

HINT: Don't forget to look at the base case first.
<pre>
def make_tree(X,y):
   ???
   return tree
</pre>

In [15]:
def select_split(X,y):
    if len(X.columns) == 1:
        return X.columns[0]
    max_col = 0
    for i in X.columns:
        gr = gain_ratio(y, X[i])
        if gr > max_col:
            max_col = gr
            best_col = i
    return best_col

In [16]:
def get_col(X,y, dictionary):
    if len(X.columns) == 0:
        return
   
    best_col = select_split(X,y)
#     print("Best Col", best_col)
    dictionary = {best_col : get_level(X,y,best_col, dictionary)}
#     print("col output dict", dictionary)
    return dictionary

In [17]:
def get_level(X,y,best_col, dictionary):
    outputdict = {}
    if len(X.columns) == 1:
        #need number
        for i in X[best_col].unique():
#             print("i:",i)
            a = X[X[best_col] == i].index
            outputdict[i] = y[a].value_counts().index[0]
#         print("number output dict", outputdict)
#         return outputdict
    else:
        #need level
        outputdict = {}
        for k in X[best_col].unique():
#             print("K:", k)
            newX = X[X[best_col] == k].drop(best_col, axis = 1)
            newY = y[newX.index]
            #outputdict = {k: get_col(newX, newY, dictionary)}
            outputdict[k] = get_col(newX,newY,dictionary)
#             outputdict = {best_col : get_col(newX, newY, dictionary)}
#             print("level output dict", outputdict)
            
            
            #return outputdict
    return outputdict

In [18]:
# d = {}
# get_col(titanic_df.drop('survived',axis=1),titanic_df['survived'], d)

In [19]:
# YOUR SOLUTION HERE
d = {}
tree = get_col(titanic_df.drop('survived',axis=1),titanic_df['survived'], d)
display(tree)

# if you want to print like me :)
def print_for_me(tree):
    import json
    import copy
    mytree = copy.deepcopy(tree)
    def fix_keys(tree):
        if type(tree) != dict:
            if type(tree) == np.int64:
                return int(tree)
        new_tree = {}
        for key in list(tree.keys()):
            if type(key) == np.int64:
                new_tree[int(key)] = tree[key]
            else:
                new_tree[key] = tree[key]
        for key in new_tree.keys():
            new_tree[key] = fix_keys(new_tree[key])
        return new_tree
    mytree = fix_keys(mytree)
    print(json.dumps(mytree, indent=4, sort_keys=True))
print_for_me(tree)

{'sex': {'female': {'pclass': {1: {'age': {20: 1,
      0: 0,
      60: 1,
      50: 1,
      10: 1,
      30: 1,
      40: 1,
      70: 1}},
    2: {'age': {20: 1, 30: 1, 10: 1, 0: 1, 40: 1, 50: 1, 60: 0}},
    3: {'age': {30: 0, 10: 1, 40: 0, 0: 1, 20: 1, 60: 1}}}},
  'male': {'age': {0: {'pclass': {1: 1, 2: 1, 3: 0}},
    30: {'pclass': {1: 0, 2: 0, 3: 0}},
    40: {'pclass': {1: 0, 2: 0, 3: 0}},
    70: {'pclass': {1: 0, 2: 0, 3: 0}},
    80: {'pclass': {1: 1}},
    20: {'pclass': {1: 0, 2: 0, 3: 0}},
    10: {'pclass': {1: 0, 2: 0, 3: 0}},
    50: {'pclass': {1: 0, 2: 0, 3: 0}},
    60: {'pclass': {1: 0, 2: 0, 3: 0}}}}}}

{
    "sex": {
        "female": {
            "pclass": {
                "1": {
                    "age": {
                        "0": 0,
                        "10": 1,
                        "20": 1,
                        "30": 1,
                        "40": 1,
                        "50": 1,
                        "60": 1,
                        "70": 1
                    }
                },
                "2": {
                    "age": {
                        "0": 1,
                        "10": 1,
                        "20": 1,
                        "30": 1,
                        "40": 1,
                        "50": 1,
                        "60": 0
                    }
                },
                "3": {
                    "age": {
                        "0": 1,
                        "10": 1,
                        "20": 1,
                        "30": 0,
                        "40": 0,
                        "60": 1
                

## Exercise 6
Modify your code above to deal with age as continuous column and include a new parameter to make tree that is the minimum number of samples required in order to consider splitting (default value should be 5). My solution was to redefine select_split to check the types. Pandas has a built in categorical object type, so it is wise to make use of it and modify our dataset as follows:

In [20]:
titanic_df.dtypes

pclass       int64
survived     int64
sex         object
age          int64
dtype: object

In [21]:
titanic_df['pclass'] = titanic_df['pclass'].astype('category')
titanic_df['survived'] = titanic_df['survived'].astype('category')
titanic_df['sex'] = titanic_df['sex'].astype('category')
titanic_df.dtypes

pclass      category
survived    category
sex         category
age            int64
dtype: object

In [22]:
titanic_df.head()

,pclass,survived,sex,age
0,1,1,female,20
1,1,1,male,0
2,1,0,female,0
3,1,0,male,30
4,1,0,female,20


In [23]:
# sex': {'female': {'pclass': {1: {'age<5.00'

In [24]:
def make_X_continuous(X):
    for i in [5,15,25,35,45,55,65,75,85]:
        newcol = "age < " + str(i)
        X.loc[X["age"] < i, newcol] = 1
        X.loc[X["age"] > i, newcol] = 0
    return X


# df.loc[df['A'] > 2, 'B'] = new_val

In [25]:
newX = make_X_continuous(titanic_df.drop("survived", axis=1))
newX

,pclass,sex,age,age < 5,age < 15,age < 25,age < 35,age < 45,age < 55,age < 65,age < 75,age < 85
0,1,female,20,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1,1,male,0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
2,1,female,0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
3,1,male,30,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0
4,1,female,20,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
1304,3,female,10,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1305,3,female,20,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1306,3,male,20,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1307,3,male,20,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [26]:
def select_split2(X, y):
    if len(X.columns) == 1:
        return X.columns[0]
#     max_col = 0
    max_col = 0
    for i in X.columns:
        gr = gain_ratio(y, X[i])
        if gr > max_col:
            max_col = gr
            best_col = i
#     print("gain ratio: ", max_col)
#     print("BEST COLUMN INSIDE SELECT SPLIT: ", best_col)
    if max_col == 0:
#         X = X[[X.columns[0]]]
#         return X.columns[0]
        return 1
    else:
#     print(X.head())
        return best_col

In [27]:
def get_col2(X, y, dictionary):
    if len(X.columns) == 0:
        return
   
    best_col = select_split2(X,y)  
    if best_col == 1:
        X = X[[X.columns[0]]]
        best_col = X.columns[0]
#     print("Best Col", best_col)
#     print("columns remaining", len(X.columns))
    dictionary = {best_col : get_level2(X, y, best_col, dictionary)}
#     print("col output dict", dictionary)
    return dictionary

In [28]:
def get_level2(X, y, best_col, dictionary):
    outputdict = {}
    if len(X.columns) == 1:
        #need number
        for i in X[best_col].unique():
#             print("i:",i)
            a = X[X[best_col] == i].index
            outputdict[i] = y[a].value_counts().index[0]
#         print("number output dict", outputdict)
#         return outputdict
    else:
        #need level
        outputdict = {}
        for k in X[best_col].unique():
#             print("K:", k)
            newX = X[X[best_col] == k].drop(best_col, axis = 1)
            newY = y[newX.index]
#             print(newX.head())
            #outputdict = {k: get_col(newX, newY, dictionary)}
            outputdict[k] = get_col2(newX, newY, dictionary)
#             outputdict = {best_col : get_col(newX, newY, dictionary)}
#             print("level output dict", outputdict)
            
            
            #return outputdict
    return outputdict

In [29]:
# a = titanic_df[[titanic_df.columns[0]]]
# a.columns[0]

In [30]:
d = {}
tree = get_col2(newX, titanic_df['survived'], d)
display(tree)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:9: RuntimeWarning: divide by zero encountered in log2
  if __name__ == '__main__':
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in double_scalars
  if __name__ == '__main__':


{'sex': {'female': {'pclass': {1: {'age < 55': {1.0: {'age < 15': {0.0: {'age < 45': {1.0: {'age < 25': {1.0: {'age': {20: 1}},
            0.0: {'age': {30: 1, 40: 1}}}},
          0.0: {'age': {50: 1}}}},
        1.0: {'age': {0: 0, 10: 1}}}},
      0.0: {'age': {60: 1, 70: 1}}}},
    2: {'age < 15': {0.0: {'age < 45': {1.0: {'age': {20: {'age < 5': {0.0: 1}},
          30: {'age < 5': {0.0: 1}},
          40: {'age < 5': {0.0: 1}}}},
        0.0: {'age': {50: 1, 60: 0}}}},
      1.0: {'age': {10: 1, 0: 1}}}},
    3: {'age < 25': {0.0: {'age < 35': {1.0: {'age': {30: 0}},
        0.0: {'age': {40: 0, 60: 1}}}},
      1.0: {'age': {10: {'age < 5': {0.0: 1}},
        0: {'age < 5': {1.0: 1}},
        20: {'age < 5': {0.0: 1}}}}}}}},
  'male': {'age < 5': {1.0: {'pclass': {1: 1, 2: 1, 3: 0}},
    0.0: {'pclass': {1: {'age < 55': {1.0: {'age': {30: {'age < 15': {0.0: {'age < 25': {0.0: {'age < 35': {1.0: {'age < 45': {1.0: {'age < 65': {1.0: {'age < 75': {1.0: {'age < 85': {1.0: 0}}}}}}}

In [31]:
# YOUR SOLUTION HERE
d = {}
tree = get_col2(newX.drop("age", axis=1),titanic_df['survived'], d)
display(tree)

# if you want to print like me :)
def print_for_me(tree):
    import json
    import copy
    mytree = copy.deepcopy(tree)
    def fix_keys(tree):
        if type(tree) != dict:
            if type(tree) == np.int64:
                return int(tree)
            else:
                return tree
        new_tree = {}
        for key in list(tree.keys()):
            if type(key) == np.int64:
                new_tree[int(key)] = tree[key]
            else:
                new_tree[key] = tree[key]
        for key in new_tree.keys():
            new_tree[key] = fix_keys(new_tree[key])
        return new_tree
    mytree = fix_keys(mytree)
    print(json.dumps(mytree, indent=4, sort_keys=True))
print_for_me(tree)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:9: RuntimeWarning: divide by zero encountered in log2
  if __name__ == '__main__':
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in double_scalars
  if __name__ == '__main__':


{'sex': {'female': {'pclass': {1: {'age < 55': {1.0: {'age < 15': {0.0: {'age < 45': {1.0: {'age < 25': {1.0: {'age < 5': {0.0: 1}},
            0.0: {'age < 5': {0.0: 1}}}},
          0.0: {'age < 5': {0.0: 1}}}},
        1.0: {'age < 5': {1.0: 0, 0.0: 1}}}},
      0.0: {'age < 5': {0.0: 1}}}},
    2: {'age < 15': {0.0: {'age < 45': {1.0: {'age < 25': {1.0: {'age < 5': {0.0: 1}},
          0.0: {'age < 35': {1.0: {'age < 5': {0.0: 1}},
            0.0: {'age < 5': {0.0: 1}}}}}},
        0.0: {'age < 5': {0.0: 1}}}},
      1.0: {'age < 5': {0.0: 1, 1.0: 1}}}},
    3: {'age < 25': {0.0: {'age < 35': {1.0: {'age < 5': {0.0: 0}},
        0.0: {'age < 5': {0.0: 0}}}},
      1.0: {'age < 15': {1.0: {'age < 5': {0.0: {'age < 35': {1.0: 1}},
          1.0: {'age < 35': {1.0: 1}}}},
        0.0: {'age < 5': {0.0: 1}}}}}}}},
  'male': {'age < 5': {1.0: {'pclass': {1: 1, 2: 1, 3: 0}},
    0.0: {'pclass': {1: {'age < 55': {1.0: {'age < 35': {1.0: {'age < 25': {0.0: {'age < 15': {0.0: {'age < 45':

{
    "sex": {
        "female": {
            "pclass": {
                "1": {
                    "age < 55": {
                        "0.0": {
                            "age < 5": {
                                "0.0": 1
                            }
                        },
                        "1.0": {
                            "age < 15": {
                                "0.0": {
                                    "age < 45": {
                                        "0.0": {
                                            "age < 5": {
                                                "0.0": 1
                                            }
                                        },
                                        "1.0": {
                                            "age < 25": {
                                                "0.0": {
                                                    "age < 5": {
                                                        "0.0": 1


## Exercise 7
Now implement a version of pruning that uses confidence intervals of the accuracy. First, let's calculate the 90% (z=1.64) confidence intervals for a node:
<pre>
def confidence_interval(y):
    ???
    return lower,upper
</pre>

Here is the formula I want you to use:
<pre>
c.i. = f +- z*sqrt( f*(1-f) / N )
</pre>
where f is the fraction of errors (1-accuracy) and N is the number of samples.

In [32]:
def confidence_interval(y):
    prediction = y.value_counts().index[0]/len(titanic_df)
    if prediction == 0:
        f = sum(y)/len(y)
        lower = f - 1.64*((f*(1-f))/len(y))**(1/2)
        upper = f + 1.64*((f*(1-f))/len(y))**(1/2)
        
    else:
        f = 1 - sum(y)/len(y)
        lower = f - 1.64*((f*(1-f))/len(y))**(1/2)
        upper = f + 1.64*((f*(1-f))/len(y))**(1/2)
        
    return lower,upper

In [33]:
# YOUR SOLUTION HERE
display(confidence_interval(titanic_df['survived']))
display(confidence_interval(pd.Series([0,0,0,1,1,1])))
display(confidence_interval(pd.Series([0,0,0])))

(0.3599470998864226, 0.403994840526106)

(0.16523640181963234, 0.8347635981803676)

(0.0, 0.0)

## Excercises 8
Now calculate the conditional confidence interval (very similar in structure to conditional entropy).

<pre>
def conditional_confidence_interval(preconditions,X,y):
    ???
    return lower,upper,ypred
</pre>

In [34]:
def conditional_confidence_interval(preconditions, X, y):
    a = X.copy()
    for i in range(len(preconditions)):
        if i != 1:
            a = a[a[preconditions[i][0]] == preconditions[i][1]]
        else:
            a = a[a[preconditions[i][0][0:3]] <= int(preconditions[i][0][4:6])]
    idx = a.index
    prediction = y.loc[idx].value_counts().index[0]
    newY = y.loc[idx]

    sum = 0 
    for i in range(len(newY)):
        if newY.iloc[i] == prediction:
            sum += 1

    accuracy = sum/len(newY)
    f = 1 - accuracy

    lower = f - 1.64*((f*(1-f))/len(newY))**(1/2)
    upper = f + 1.64*((f*(1-f))/len(newY))**(1/2)
    
    return lower, upper

In [35]:
# YOUR SOLUTION HERE
display(conditional_confidence_interval([('sex', 'male'), ('age<75.00', 'True'), ('pclass', 2)],
                                        titanic_df.drop('survived',axis=1),titanic_df['survived']))


(0.10188940536593324, 0.1905082554527803)

In [36]:
# YOUR SOLUTION HERE
display(conditional_confidence_interval([('sex', 'female'), ('age<50.00', 'True'), ('pclass', 1)],
                                        titanic_df.drop('survived',axis=1),titanic_df['survived']))



(0.005695984971025896, 0.053563274288233445)

In [37]:
a = titanic_df[(titanic_df.sex == "female") & (titanic_df.age <= 50) & (titanic_df.pclass == 1)]
a["survived"].value_counts()

1    131
0      4
Name: survived, dtype: int64

In [38]:
accuracy = 131/135
f = 1 - accuracy
lower = f - 1.64*((f*(1-f))/len(a))**(1/2)
upper = f + 1.64*((f*(1-f))/len(a))**(1/2)
print(lower, upper)

0.005695984971025896 0.053563274288233445


## Excercises 9
Now we can put together a pruning algorithm. Please implement statistical pruning, but here a short description of reduced-error pruning as well. Original source of some text: [http://www.cs.bc.edu/~alvarez/ML/statPruning.html](http://www.cs.bc.edu/~alvarez/ML/statPruning.html)

**Reduced-Error Pruning**: One approach to pruning is to withhold a portion of the available labeled data for validation. The validation set is not used during training. Once training has been completed, testing is carried out over the validation set. If the error rate of the original decision tree over the validation set exceeds the error rate of a pruned version of the tree (obtained by replacing a near-leaf node with its child leaves by a single leaf node), then the pruning operation is carried out. Reduced error pruning can reduce overfitting, but it also reduces the amount of data available for training.

**Statistical Pruning**: C4.5 uses a pruning technique based on statistical confidence estimates. This technique has the advantage that it allows all of the available labeled data to be used for training.

The heart of the statistical pruning technique is the calculation of a confidence interval for the error rate. In brief, one starts from an observed error rate f measured over the set of N training instances. In order to decide whether to replace a near-leaf node and its child leaves by a single leaf node, C4.5 compares the upper limits of the error confidence intervals for the two trees. For the unpruned tree, the upper error estimate is calculated as a weighted average over its child leaves (what you implemented above). Whichever tree has a lower estimated upper limit on the error rate "wins" and is selected.

<pre>
def generate_rules(tree,rules=[]):
   ???
   
def prune_rules(rules,X,y):
   return new_rules # (sorted by accuracy)
</pre>

In [39]:
def generate_rules(tree):
    rules = []
    if type(tree) != dict:
        return [[tree]]
    for key in tree.keys():
        for xv in tree[key]:
            new_rules = generate_rules(tree[key][xv])
            for rule in new_rules:
                rules.append([(key, xv)] + rule)
    return rules
    

In [40]:
# YOUR SOLUTION HERE
d = {}
tree = get_col2(newX.drop("age", axis=1),titanic_df['survived'], d)
rules = generate_rules(tree)
print('Original rules:')
for rule in rules:
    print(rule)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:9: RuntimeWarning: divide by zero encountered in log2
  if __name__ == '__main__':
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in double_scalars
  if __name__ == '__main__':


Original rules:
[('sex', 'female'), ('pclass', 1), ('age < 55', 1.0), ('age < 15', 0.0), ('age < 45', 1.0), ('age < 25', 1.0), ('age < 5', 0.0), 1]
[('sex', 'female'), ('pclass', 1), ('age < 55', 1.0), ('age < 15', 0.0), ('age < 45', 1.0), ('age < 25', 0.0), ('age < 5', 0.0), 1]
[('sex', 'female'), ('pclass', 1), ('age < 55', 1.0), ('age < 15', 0.0), ('age < 45', 0.0), ('age < 5', 0.0), 1]
[('sex', 'female'), ('pclass', 1), ('age < 55', 1.0), ('age < 15', 1.0), ('age < 5', 1.0), 0]
[('sex', 'female'), ('pclass', 1), ('age < 55', 1.0), ('age < 15', 1.0), ('age < 5', 0.0), 1]
[('sex', 'female'), ('pclass', 1), ('age < 55', 0.0), ('age < 5', 0.0), 1]
[('sex', 'female'), ('pclass', 2), ('age < 15', 0.0), ('age < 45', 1.0), ('age < 25', 1.0), ('age < 5', 0.0), 1]
[('sex', 'female'), ('pclass', 2), ('age < 15', 0.0), ('age < 45', 1.0), ('age < 25', 0.0), ('age < 35', 1.0), ('age < 5', 0.0), 1]
[('sex', 'female'), ('pclass', 2), ('age < 15', 0.0), ('age < 45', 1.0), ('age < 25', 0.0), ('age <

In [41]:
def conditional_confidence_interval2(preconditions, X, y):
    a = X.copy()
    for i in range(len(preconditions)):
        if "age" not in preconditions[i]:
            a = a[a[preconditions[i][0]] == preconditions[i][1]]
        else:
            a = a[a[preconditions[i][0][0:3]] <= int(preconditions[i][0][4:6])]
    idx = a.index
    prediction = y.loc[idx].value_counts().index[0]
    newY = y.loc[idx]
 
    sum = 0
    for i in range(len(newY)):
        if newY.iloc[i] == prediction:
            sum += 1
 
    accuracy = sum/len(newY)
    f = 1 - accuracy
 
    lower = f - 1.64*((f*(1-f))/len(newY))**(1/2)
    upper = f + 1.64*((f*(1-f))/len(newY))**(1/2)
   
    return lower, upper

In [42]:

def prune_rules(rules, X, y):
    outputrules = rules.copy()
    
    for i in range(len(rules)):
        for j in range(2, len(rules[i])):
            noprune = conditional_confidence_interval2(rules[i][:-j+1], X, y)
            prune = conditional_confidence_interval2(rules[i][:-j], X, y)
            if prune[1] <= noprune[1]:
                outputrules[i] = (rules[i][:-j], prune[1])
            else:
                outputrules[i] = (rules[i][:-j+1], noprune[1])
                break
    return outputrules
       

new_rules = prune_rules(rules,newX.drop('age',axis=1),titanic_df['survived']) 

sorted1 = sorted(new_rules, key=lambda x: x[1])
sorted1

[([('sex', 'female'),
   ('pclass', 1),
   ('age < 55', 1.0),
   ('age < 15', 1.0),
   ('age < 5', 1.0)],
  0.0),
 ([('sex', 'female'),
   ('pclass', 1),
   ('age < 55', 1.0),
   ('age < 15', 1.0),
   ('age < 5', 0.0)],
  0.0),
 ([('sex', 'female'), ('pclass', 2), ('age < 15', 1.0), ('age < 5', 1.0)],
  0.0),
 ([('sex', 'male'), ('age < 5', 1.0), ('pclass', 1)], 0.0),
 ([('sex', 'male'), ('age < 5', 1.0), ('pclass', 2)], 0.0),
 ([('sex', 'female'),
   ('pclass', 1),
   ('age < 55', 1.0),
   ('age < 15', 0.0),
   ('age < 45', 1.0),
   ('age < 25', 0.0)],
  0.04377126515583939),
 ([('sex', 'female'),
   ('pclass', 1),
   ('age < 55', 1.0),
   ('age < 15', 0.0),
   ('age < 45', 1.0)],
  0.04383185526626483),
 ([('sex', 'female'), ('pclass', 1), ('age < 55', 1.0), ('age < 15', 0.0)],
  0.04877755756143441),
 ([('sex', 'female'), ('pclass', 1)], 0.059742522193751216),
 ([('sex', 'male'),
   ('age < 5', 0.0),
   ('pclass', 2),
   ('age < 35', 0.0),
   ('age < 55', 1.0)],
  0.0870811670536762

# Exercise 10
Now let's run our algorithm on our activity dataset. What is the resulting tree before and after pruning? Was anything pruned? What are the rules after pruning?

In [43]:
activity = pd.read_csv("activity.csv")
activity.head()
 
#before
d={}
tree = get_col2(activity.drop("activity", axis=1), activity.activity, d)
tree
rules = generate_rules(tree)
rules

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in double_scalars
  This is separate from the ipykernel package so we can avoid doing imports until


[[('party', 'Yes'), ('deadline', 'Soon'), ('age', 22), ('lazy', 'No'), 'Bar'],
 [('party', 'Yes'),
  ('deadline', 'Soon'),
  ('age', 20),
  ('lazy', 'Yes'),
  'Party'],
 [('party', 'Yes'),
  ('deadline', 'Soon'),
  ('age', 20),
  ('lazy', 'No'),
  'Party'],
 [('party', 'Yes'),
  ('deadline', 'Soon'),
  ('age', 21),
  ('lazy', 'No'),
  'Party'],
 [('party', 'Yes'),
  ('deadline', 'Soon'),
  ('age', 21),
  ('lazy', 'Yes'),
  'Party'],
 [('party', 'Yes'), ('deadline', 'None'), ('age', 21), ('lazy', 'No'), 'Bar'],
 [('party', 'Yes'),
  ('deadline', 'None'),
  ('age', 20),
  ('lazy', 'No'),
  'Party'],
 [('party', 'Yes'),
  ('deadline', 'Urgent'),
  ('age', 21),
  ('lazy', 'No'),
  'Party'],
 [('party', 'Yes'),
  ('deadline', 'Urgent'),
  ('age', 20),
  ('lazy', 'Yes'),
  'Study'],
 [('party', 'Yes'),
  ('deadline', 'Urgent'),
  ('age', 22),
  ('lazy', 'Yes'),
  'TV'],
 [('party', 'No'), ('deadline', 'Soon'), ('age', 20), ('lazy', 'Yes'), 'TV'],
 [('party', 'No'), ('deadline', 'Soon'), ('ag

In [44]:
#after pruning
def conditional_confidence_interval3(preconditions, X, y):
    a = X.copy()
    for i in range(len(preconditions)-1):
        a = a[a[preconditions[i][0]] == preconditions[i][1]]
    idx = a.index
    prediction = y.loc[idx].value_counts().index[0]
    newY = y.loc[idx]
 
    sum = 0
    for i in range(len(newY)):
        if newY.iloc[i] == prediction:
            sum += 1
 
    accuracy = sum/len(newY)
    f = 1 - accuracy
 
    lower = f - 1.64*((f*(1-f))/len(newY))**(1/2)
    upper = f + 1.64*((f*(1-f))/len(newY))**(1/2)
   
    return lower, upper
 
def prune_rules2(rules, X, y):
    outputrules = rules.copy()
   
    
    for i in range(len(rules)):
        for j in range(2, len(rules[i])):
            noprune = conditional_confidence_interval3(rules[i][:-j+1], X, y)
            prune = conditional_confidence_interval3(rules[i][:-j], X, y)
            if prune[1] <= noprune[1]:
                outputrules[i] = (rules[i][:-j], prune[1])
            else:
                outputrules[i] = (rules[i][:-j+1], noprune[1])
                break
    return outputrules

In [45]:
#after pruning
rules = generate_rules(tree)
 
new_rules = prune_rules2(rules,activity.drop('activity',axis=1),activity['activity']) 
 
sorted1 = sorted(new_rules, key=lambda x: x[1])
sorted1

[([('party', 'Yes'), ('deadline', 'Soon'), ('age', 20), ('lazy', 'Yes')], 0.0),
 ([('party', 'Yes'), ('deadline', 'Soon'), ('age', 20), ('lazy', 'No')], 0.0),
 ([('party', 'Yes'), ('deadline', 'None'), ('age', 21), ('lazy', 'No')], 0.0),
 ([('party', 'Yes'), ('deadline', 'None'), ('age', 20), ('lazy', 'No')], 0.0),
 ([('party', 'Yes'), ('deadline', 'Urgent'), ('age', 20), ('lazy', 'Yes')],
  0.0),
 ([('party', 'Yes'), ('deadline', 'Urgent'), ('age', 22), ('lazy', 'Yes')],
  0.0),
 ([('party', 'No'), ('deadline', 'Soon'), ('age', 23), ('lazy', 'Yes')], 0.0),
 ([('party', 'No'), ('deadline', 'Urgent'), ('lazy', 'Yes')], 0.0),
 ([('party', 'No'), ('deadline', 'Urgent'), ('lazy', 'No')], 0.0),
 ([('party', 'No'), ('deadline', 'None'), ('lazy', 'Yes')], 0.0),
 ([('party', 'Yes'), ('deadline', 'Soon'), ('age', 22)], 0.5376585786375068),
 ([('party', 'Yes'), ('deadline', 'Soon'), ('age', 21)], 0.5376585786375068),
 ([('party', 'Yes'), ('deadline', 'Soon'), ('age', 21)], 0.5376585786375068),
 

## Exercise 11
Considering the titanic dataset, what is the most important feature? How does the test set accuracy compare to the accuracy using naive Bayes from last lab? Does pruning help this accuracy?

HINT: It makes sense to run the combination of train+test at least 20 times to make sure we aren't just getting lucky with a single run. 

In [46]:
part11X = make_X_continuous(titanic_df)
part11X = part11X.drop("age", axis=1)

In [47]:
part11X.head()

,pclass,survived,sex,age < 5,age < 15,age < 25,age < 35,age < 45,age < 55,age < 65,age < 75,age < 85
0,1,1,female,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1,1,1,male,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
2,1,0,female,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
3,1,0,male,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0
4,1,0,female,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [48]:
d={}
tree = get_col2(part11X.drop("survived", axis=1), part11X.survived, d)
rules = generate_rules(tree)
new_rules = prune_rules2(rules, part11X.drop('survived',axis=1), part11X['survived']) 

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:9: RuntimeWarning: divide by zero encountered in log2
  if __name__ == '__main__':
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in double_scalars
  if __name__ == '__main__':


In [49]:
def get_accuracy(rules, X, y):
    X2 = X.copy()
    X2 = X2.drop('survived', axis=1)
    X["prediction"] = 0

    for i in range(len(rules)):
        for j in range(len(rules[i]) - 1):
            X2 = X[X[rules[i][j][0]] == rules[i][j][1]]
        
        test = X2.index.tolist()
        pred = rules[i][-1]
        
        for k in test:
            X.loc[k, "prediction"] = pred

    total_right = 0
    for a in range(len(X2)):
        if X.loc[a, "prediction"] == X.loc[a, "survived"]:
            total_right +=1
   
    return total_right / len(X)
 
get_accuracy(rules, part11X, titanic_df['survived'])

0.5500381970970206

## Exercise 12
So pruned helps somewhat in this example, but we are mostly saved the need to pruning because we are only considering three variables and we are only considering age once in the tree. Not much chance of overfitting. What happens if we include the additional columns of fare, embarked, cabin, home.dest, parch, and sibsp?

HINT: It makes sense to run the combination of train+test at least 20 times to make sure we aren't just getting lucky with a single run. 

In [50]:
import pandas as pd
titanic_df = pd.read_csv(
    "https://raw.githubusercontent.com/dlsun/data-science-book/master/data/titanic.csv"
)
titanic_df.head()
features = ['pclass','survived','sex','age','fare','embarked','cabin','home.dest','parch','sibsp']
titanic_df = titanic_df.loc[:,features]
titanic_df.loc[:,'survived']=titanic_df['survived'].astype('category')
titanic_df.loc[:,'pclass']=titanic_df['pclass'].fillna(titanic_df['pclass'].mode().values[0]).astype('category')
titanic_df.loc[:,'age']=titanic_df['age'].fillna(titanic_df['age'].median())
titanic_df.loc[:,'fare']=titanic_df['fare'].fillna(titanic_df['fare'].median())
titanic_df.loc[:,'embarked']=titanic_df['embarked'].fillna(titanic_df['embarked'].mode().values[0]).astype('category')
titanic_df.loc[:,'cabin']=titanic_df['cabin'].fillna(titanic_df['cabin'].mode().values[0]).astype('category')
titanic_df.loc[:,'home.dest']=titanic_df['home.dest'].fillna(titanic_df['home.dest'].mode().values[0]).astype('category')
titanic_df.loc[:,'parch']=titanic_df['parch'].fillna(titanic_df['parch'].mode().values[0]).astype('category')
titanic_df.loc[:,'sibsp']=titanic_df['sibsp'].fillna(titanic_df['sibsp'].mode().values[0]).astype('category')
titanic_df.loc[:,'sex']=titanic_df['sex'].fillna(titanic_df['sex'].mode().values[0]).astype('category')
display(titanic_df.dtypes)
display(titanic_df)

pclass       category
survived     category
sex          category
age           float64
fare          float64
embarked     category
cabin        category
home.dest    category
parch        category
sibsp        category
dtype: object

,pclass,survived,sex,age,fare,embarked,cabin,home.dest,parch,sibsp
0,1,1,female,29.0000,211.3375,S,B5,"St Louis, MO",0,0
1,1,1,male,0.9167,151.5500,S,C22 C26,"Montreal, PQ / Chesterville, ON",2,1
2,1,0,female,2.0000,151.5500,S,C22 C26,"Montreal, PQ / Chesterville, ON",2,1
3,1,0,male,30.0000,151.5500,S,C22 C26,"Montreal, PQ / Chesterville, ON",2,1
4,1,0,female,25.0000,151.5500,S,C22 C26,"Montreal, PQ / Chesterville, ON",2,1
...,...,...,...,...,...,...,...,...,...,...
1304,3,0,female,14.5000,14.4542,C,C23 C25 C27,"New York, NY",0,1
1305,3,0,female,28.0000,14.4542,C,C23 C25 C27,"New York, NY",0,1
1306,3,0,male,26.5000,7.2250,C,C23 C25 C27,"New York, NY",0,0
1307,3,0,male,27.0000,7.2250,C,C23 C25 C27,"New York, NY",0,0


In [51]:
d={}
tree = get_col2(titanic_df.drop("survived", axis=1), titanic_df.survived, d)
rules = generate_rules(tree)

get_accuracy(rules, titanic_df, titanic_df['survived'])

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:9: RuntimeWarning: divide by zero encountered in log2
  if __name__ == '__main__':
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in double_scalars
  if __name__ == '__main__':


0.0038197097020626434

## Exercise 13

What modifications can you make to the pruning algorithm such that it trims more often? Did this improve the results?

In [52]:
# YOUR SOLUTION HERE

## Exercise 14

For your final exercise, I would like you to write and apply two functions to both the activity dataset and the titanic dataset. Try to create a nice output in a table format that summarizes how each of the three scores vary for each dataset and for each possible value of pos_label.

<pre>
def precision(y,ypred,pos_label):
   ???
   return p
   
def recall(y,ypred,pos_label):
   ???
   return r
   
def f1(y,ypred,pos_label):
   ???
   return f
</pre>

In [53]:
def precision(X, y, pos_label):
    d={}
    if "activity" in X.columns:
        tree = get_col2(X.drop("activity", axis=1), y, d)
    elif "survived" in X.columns:
        tree = get_col2(X.drop("survived", axis=1), y, d)
    rules = generate_rules(tree)
        
    X2 = X.copy()
    X["prediction"] = 0

    for i in range(len(rules)):
        for j in range(len(rules[i]) - 1):
            X2 = X[X[rules[i][j][0]] == rules[i][j][1]]
        
        test = X2.index.tolist()
        pred = rules[i][-1]
        
        for k in test:
            X.loc[k, "prediction"] = pred


    total_right = 0
    for a in X.index.tolist():
        if "activity" in X.columns:
            if X.loc[a, "prediction"] == pos_label:
                if X.loc[a, "prediction"] == X.loc[a, "activity"]:
                    total_right += 1
        if "survived" in X.columns:
            if X.loc[a, "prediction"] == pos_label:
                if X.loc[a, "prediction"] == X.loc[a, "survived"]:
                    total_right +=1
   
    return total_right / len(X)

In [54]:
def recall(X, y, pos_label):
    d={}
    if "activity" in X.columns:
        tree = get_col2(X.drop("activity", axis=1), y, d)
    elif "survived" in X.columns:
        tree = get_col2(X.drop("survived", axis=1), y, d)
    rules = generate_rules(tree)
        
    X2 = X.copy()
    X["prediction"] = 0

    for i in range(len(rules)):
        for j in range(len(rules[i]) - 1):
            X2 = X[X[rules[i][j][0]] == rules[i][j][1]]
        
        test = X2.index.tolist()
        pred = rules[i][-1]
        
        for k in test:
            X.loc[k, "prediction"] = pred

    if "activity" in X.columns:
        filtered_X = X[X.activity == pos_label]
    elif "survived" in X.columns:
        filtered_X = X[X.survived == pos_label]
        
    total_right = 0
    for a in filtered_X.index.tolist():
        if "activity" in X.columns:
            if filtered_X.loc[a, "prediction"] == filtered_X.loc[a, "activity"]:
                total_right += 1
        if "survived" in X.columns:
            if filtered_X.loc[a, "prediction"] == filtered_X.loc[a, "survived"]:
                total_right +=1
   
    return total_right / len(filtered_X)

In [55]:
def f1(X, y, pos_label):
    p = precision(X, y, pos_label)
    r = recall(X, y, pos_label)
    if p+r == 0:
        return 0.0
    return (2 * p * r) / (p + r)

In [56]:
print("Activity Dataset: ")
for i in activity.activity.unique():
    print("   ", i)
    print("       Precision: ", precision(activity, activity.activity, i))
    print("       Recall: ", recall(activity, activity.activity, i))
    print("       F1: ", f1(activity, activity.activity, i))
    
    
print("Titanic Dataset: ")
for i in titanic_df.survived.unique():
    print("   ", i)
    print("       Precision: ", precision(titanic_df, titanic_df.survived, i))
    print("       Recall: ", recall(titanic_df, titanic_df.survived, i))
    print("       F1: ", f1(titanic_df, titanic_df.survived, i))

Activity Dataset: 
    Party


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in double_scalars
  This is separate from the ipykernel package so we can avoid doing imports until


       Precision:  0.0
       Recall:  0.0
       F1:  0.0
    TV
       Precision:  0.26666666666666666
       Recall:  0.8571428571428571
       F1:  0.4067796610169491
    Bar
       Precision:  0.0
       Recall:  0.0
       F1:  0.0
    Study
       Precision:  0.17777777777777778
       Recall:  0.5
       F1:  0.26229508196721313
Titanic Dataset: 
    1


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:9: RuntimeWarning: divide by zero encountered in log2
  if __name__ == '__main__':
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in double_scalars
  if __name__ == '__main__':


       Precision:  0.03284950343773873
       Recall:  0.184
       F1:  0.05574657572853841
    0
       Precision:  0.6035141329258976
       Recall:  0.8936959208899876
       F1:  0.7204842332185653
